stage a: preprocesing * 3 embedings using one basline model (linear SVM)
stage b : pick best pipeline
stage c : fix and compare best pipeline


In [3]:
pip install pyarabic

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install gensim


Note: you may need to restart the kernel to use updated packages.


In [1]:
import re
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from gensim.models import FastText

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import AdamW

from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

from pyarabic.araby import strip_tashkeel, strip_tatweel


load data


In [2]:
df = pd.read_csv("tweets.csv")
df = df.dropna(subset=["text", "class"]).reset_index(drop=True)

df["class"] = df["class"].astype(str).str.strip()

label_map = {"Negative": 0, "Neutral": 1, "Positive": 2}
df["label"] = df["class"].map(label_map)

# drop any unexpected labels (e.g., "class")
df = df.dropna(subset=["label"]).reset_index(drop=True)
df["label"] = df["label"].astype(int)

print("Shape:", df.shape)
print(df["class"].value_counts())
df.head()


Shape: (1193, 4)
class
Positive    398
Negative    398
Neutral     397
Name: count, dtype: int64


,text,class,id,label
0,يسعد صباحك يارب,Positive,1728269882452844808,2
1,البادية النيابية تزور بنك تنمية المدن والقرى ت...,Neutral,1733894221587718209,1
2,ناس تخاف تستحي سبحان مغير الأحوال بعد الهجوم ا...,Negative,1728031560039887079,0
3,فريق مبادرة الاردنيدعمفلسطين شباب اردنيون يثمن...,Positive,1730957855929270642,2
4,صباح الخير عاجل الاردن تسحب سفيرها الآراضي ال...,Neutral,1719751382712844483,1


tran test spplit


In [3]:
X = df["text"].astype(str).tolist()
y = df["label"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train:", len(X_train), "Test:", len(X_test))


Train: 954 Test: 239


In [4]:
def remove_urls_mentions_hashtags(text: str) -> str:
    text = re.sub(r"http\S+|www\S+", " ", text)
    text = re.sub(r"@\w+", " ", text)
    text = re.sub(r"#\w+", " ", text)
    return text

def remove_non_arabic(text: str) -> str:
    return re.sub(r"[^\u0600-\u06FF\s]", " ", text)

def normalize_spaces(text: str) -> str:
    return re.sub(r"\s+", " ", text).strip()

def arabic_normalize(text: str) -> str:
    text = strip_tashkeel(text)
    text = strip_tatweel(text)
    text = text.replace("أ", "ا").replace("إ", "ا").replace("آ", "ا")
    text = text.replace("ى", "ي")
    text = text.replace("ة", "ه")
    return text

# P0: Raw
def preprocess_p0(text: str) -> str:
    return normalize_spaces(str(text))

# P1: Regex cleaning
def preprocess_p1(text: str) -> str:
    text = str(text)
    text = remove_urls_mentions_hashtags(text)
    text = remove_non_arabic(text)
    return normalize_spaces(text)

# P2: Normalization
def preprocess_p2(text: str) -> str:
    text = preprocess_p1(text)
    text = arabic_normalize(text)
    return normalize_spaces(text)


**P0/P1/P2 train & test sets . (3 preprocessed datasets)**

In [5]:
X_train_p0 = [preprocess_p0(t) for t in X_train]
X_test_p0  = [preprocess_p0(t) for t in X_test]

X_train_p1 = [preprocess_p1(t) for t in X_train]
X_test_p1  = [preprocess_p1(t) for t in X_test]

X_train_p2 = [preprocess_p2(t) for t in X_train]
X_test_p2  = [preprocess_p2(t) for t in X_test]

print("Sample P0:", X_train_p0[0])
print("Sample P1:", X_train_p1[0])
print("Sample P2:", X_train_p2[0])


Sample P0: الطلعة عالحدود بتشتت العدو وبتخليه يعرف انه بدل غزة الف غزة بتخوف المستطونين وبتلعب عالحرب النفسية بتثبط عزيمة الاحتلال وبتقوي عزيمة شعبنا بغزة تعتبر سلاح فعال جدا بتشتيت قوى العدو اتجاه غزة الطلعة كتير مهمة لازم الاردن كلها توقف عالحدود
Sample P1: الطلعة عالحدود بتشتت العدو وبتخليه يعرف انه بدل غزة الف غزة بتخوف المستطونين وبتلعب عالحرب النفسية بتثبط عزيمة الاحتلال وبتقوي عزيمة شعبنا بغزة تعتبر سلاح فعال جدا بتشتيت قوى العدو اتجاه غزة الطلعة كتير مهمة لازم الاردن كلها توقف عالحدود
Sample P2: الطلعه عالحدود بتشتت العدو وبتخليه يعرف انه بدل غزه الف غزه بتخوف المستطونين وبتلعب عالحرب النفسيه بتثبط عزيمه الاحتلال وبتقوي عزيمه شعبنا بغزه تعتبر سلاح فعال جدا بتشتيت قوي العدو اتجاه غزه الطلعه كتير مهمه لازم الاردن كلها توقف عالحدود


**eval basline model**

In [6]:
def eval_clf(name, clf, Xtr, Xte):
    clf.fit(Xtr, y_train)
    pred = clf.predict(Xte)
    acc = accuracy_score(y_test, pred)
    mf1 = f1_score(y_test, pred, average="macro")
    print(f"\n{name}")
    print("Accuracy:", acc)
    print("Macro-F1:", mf1)
    return acc, mf1


**Stage A 9 expetiment (baseline model fixed: Linear SVM)

In [7]:
def tfidf_features(train_texts, test_texts):
    vec = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
    Xtr = vec.fit_transform(train_texts)
    Xte = vec.transform(test_texts)
    return Xtr, Xte


# ** Traditional embedding Run SVM + TF-IDF on P0/P1/P2**

In [8]:
results = []

for pname, tr, te in [
    ("P0 Raw", X_train_p0, X_test_p0),
    ("P1 Regex", X_train_p1, X_test_p1),
    ("P2 Norm", X_train_p2, X_test_p2),
]:
    Xtr, Xte = tfidf_features(tr, te)
    acc, mf1 = eval_clf(f"Stage A: SVM + TF-IDF ({pname})", LinearSVC(), Xtr, Xte)
    results.append({"Preprocess": pname, "Embedding": "TF-IDF", "BaselineModel": "SVM", "Accuracy": acc, "MacroF1": mf1})



Stage A: SVM + TF-IDF (P0 Raw)
Accuracy: 0.602510460251046
Macro-F1: 0.6025320591358326

Stage A: SVM + TF-IDF (P1 Regex)
Accuracy: 0.602510460251046
Macro-F1: 0.6025320591358326

Stage A: SVM + TF-IDF (P2 Norm)
Accuracy: 0.6401673640167364
Macro-F1: 0.6402701539123883


training fast text on train(inpependent embedding)

In [9]:
def simple_tokenize(text: str):
    return text.split()

train_tokens = [simple_tokenize(t) for t in X_train_p2]  # train on normalized train text

ft = FastText(
    sentences=train_tokens,
    vector_size=100,
    window=5,
    min_count=1,
    sg=1,
    workers=1,
    epochs=20
)

print("FastText trained.")


FastText trained.


# **fastText avg vectors**

In [10]:
def doc_vector_avg(tokens, model, dim=100):
    vecs = [model.wv[w] for w in tokens if w in model.wv]
    if len(vecs) == 0:
        return np.zeros(dim)
    return np.mean(vecs, axis=0)

def fasttext_features(train_texts, test_texts, model, dim=100):
    Xtr = np.vstack([doc_vector_avg(simple_tokenize(t), model, dim) for t in train_texts])
    Xte = np.vstack([doc_vector_avg(simple_tokenize(t), model, dim) for t in test_texts])
    return Xtr, Xte


# **Stage A / independent embeddings :Run SVM + FastTextAvg on P0/P1/P2 (3 experiments)**

In [11]:
for pname, tr, te in [
    ("P0 Raw", X_train_p0, X_test_p0),
    ("P1 Regex", X_train_p1, X_test_p1),
    ("P2 Norm", X_train_p2, X_test_p2),
]:
    Xtr, Xte = fasttext_features(tr, te, ft, dim=100)
    acc, mf1 = eval_clf(f"Stage A: SVM + FastTextAvg ({pname})", LinearSVC(), Xtr, Xte)
    results.append({"Preprocess": pname, "Embedding": "FastTextAvg", "BaselineModel": "SVM", "Accuracy": acc, "MacroF1": mf1})



Stage A: SVM + FastTextAvg (P0 Raw)
Accuracy: 0.5648535564853556
Macro-F1: 0.563958662761058

Stage A: SVM + FastTextAvg (P1 Regex)
Accuracy: 0.5648535564853556
Macro-F1: 0.5640509725016768

Stage A: SVM + FastTextAvg (P2 Norm)
Accuracy: 0.5271966527196653
Macro-F1: 0.527185078557067


# ** load AraBert encoder for CLS embeding depending emb**

In [12]:
MODEL_NAME = "aubmindlab/bert-base-arabertv02"

bert_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert_encoder = AutoModel.from_pretrained(MODEL_NAME)
bert_encoder.eval()

device = "cpu" 
bert_encoder.to(device)

print("AraBERT encoder loaded on", device)


AraBERT encoder loaded on cpu


/opt/anaconda3/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


# **CLS embedding fun feature based**

In [13]:
def bert_cls_embeddings(texts, tokenizer, model, max_len=64, batch_size=16, device="cpu"):
    all_vecs = []
    model.eval()
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
          batch = texts[i:i+batch_size]
          enc = tokenizer(batch, padding=True, truncation=True, max_length=max_len, return_tensors="pt")
          enc = {k: v.to(device) for k, v in enc.items()}
          out = model(**enc)
          cls = out.last_hidden_state[:, 0, :].cpu().numpy()
          all_vecs.append(cls)
    return np.vstack(all_vecs)


Stage A / Dependent embedding: AraBERT-CLS + SVM (3 runs)

In [14]:
for pname, tr, te in [
    ("P0 Raw", X_train_p0, X_test_p0),
    ("P1 Regex", X_train_p1, X_test_p1),
    ("P2 Norm", X_train_p2, X_test_p2),
]:
    Xtr = bert_cls_embeddings(tr, bert_tokenizer, bert_encoder, device=device)
    Xte = bert_cls_embeddings(te, bert_tokenizer, bert_encoder, device=device)
    acc, mf1 = eval_clf(f"Stage A: SVM + AraBERT-CLS ({pname})", LinearSVC(), Xtr, Xte)
    results.append({"Preprocess": pname, "Embedding": "AraBERT-CLS", "BaselineModel": "SVM", "Accuracy": acc, "MacroF1": mf1})



Stage A: SVM + AraBERT-CLS (P0 Raw)
Accuracy: 0.5104602510460251
Macro-F1: 0.5079463814772377

Stage A: SVM + AraBERT-CLS (P1 Regex)
Accuracy: 0.502092050209205
Macro-F1: 0.5006172591311909

Stage A: SVM + AraBERT-CLS (P2 Norm)
Accuracy: 0.5564853556485355
Macro-F1: 0.5555456108728531


# **Stage A results table (9 experiments)**

In [15]:
results_df = pd.DataFrame(results)
results_df


,Preprocess,Embedding,BaselineModel,Accuracy,MacroF1
0,P0 Raw,TF-IDF,SVM,0.602510,0.602532
1,P1 Regex,TF-IDF,SVM,0.602510,0.602532
2,P2 Norm,TF-IDF,SVM,0.640167,0.640270
3,P0 Raw,FastTextAvg,SVM,0.564854,0.563959
4,P1 Regex,FastTextAvg,SVM,0.564854,0.564051
5,P2 Norm,FastTextAvg,SVM,0.527197,0.527185
6,P0 Raw,AraBERT-CLS,SVM,0.510460,0.507946
7,P1 Regex,AraBERT-CLS,SVM,0.502092,0.500617
8,P2 Norm,AraBERT-CLS,SVM,0.556485,0.555546


# **Stage b pick best pipeline**

In [16]:
best_row = results_df.sort_values("MacroF1", ascending=False).iloc[0]
print("best pieline from stage A :")
best_row


best pieline from stage A :


Preprocess        P2 Norm
Embedding          TF-IDF
BaselineModel         SVM
Accuracy         0.640167
MacroF1           0.64027
Name: 2, dtype: object

# **Syage C fix best preprocesing +embeding and compare models**

In [17]:
best_pre = best_row["Preprocess"]
best_emb = best_row["Embedding"]
if best_pre == "P0 Raw":
    Xtr_text, Xte_text = X_train_p0, X_test_p0
elif best_pre == "P1 Regex":
    Xtr_text, Xte_text = X_train_p1, X_test_p1
else:
    Xtr_text, Xte_text = X_train_p2, X_test_p2
print("Stage C fixed preprocessing:", best_pre)
print("Stage C fixed embedding:", best_emb)


Stage C fixed preprocessing: P2 Norm
Stage C fixed embedding: TF-IDF


build fixed features for Stage C based on best embedding

In [19]:
if best_emb == "TF-IDF":
    vec_stagec = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
    Xtr_best = vec_stagec.fit_transform(Xtr_text)
    Xte_best = vec_stagec.transform(Xte_text)

elif best_emb == "FastTextAvg":
    Xtr_best, Xte_best = fasttext_features(Xtr_text, Xte_text, ft, dim=100)

else:  #AraBert-CLS
    Xtr_best = bert_cls_embeddings(Xtr_text, bert_tokenizer, bert_encoder, device=device)
    Xte_best = bert_cls_embeddings(Xte_text, bert_tokenizer, bert_encoder, device=device)

print("Stage C feature shapes:", Xtr_best.shape, Xte_best.shape)


Stage C feature shapes: (954, 5000) (239, 5000)


Stage C traditional ML comparison pipewple is fixed

In [22]:
trad_models = [
    ("SVM", LinearSVC()),
    ("LogReg", LogisticRegression(max_iter=500)),
    ("DecisionTree", DecisionTreeClassifier(random_state=42)),
]

trad_rows = []
for name, clf in trad_models:
    clf.fit(Xtr_best, y_train)
    pred = clf.predict(Xte_best)
    acc = accuracy_score(y_test, pred)
    mf1 = f1_score(y_test, pred, average="macro")
    trad_rows.append({"Model": name, "Accuracy": acc, "MacroF1": mf1})

final_df = pd.DataFrame(trad_rows).sort_values("MacroF1", ascending=False).reset_index(drop=True)
final_df


,Model,Accuracy,MacroF1
0,SVM,0.640167,0.640270
1,LogReg,0.635983,0.636162
2,DecisionTree,0.548117,0.541520


classification report for best traditional ML model

In [23]:
best_trad_model = final_df.iloc[0]["Model"]

if best_trad_model == "SVM":
    clf = LinearSVC()
elif best_trad_model == "LogReg":
    clf = LogisticRegression(max_iter=500)
else:
    clf = DecisionTreeClassifier(random_state=42)

clf.fit(Xtr_best, y_train)
pred = clf.predict(Xte_best)

print("Best Traditional ML:", best_trad_model)
print(classification_report(y_test, pred, target_names=["Negative","Neutral","Positive"]))


Best Traditional ML: SVM
              precision    recall  f1-score   support

    Negative       0.60      0.62      0.61        80
     Neutral       0.64      0.68      0.66        79
    Positive       0.68      0.61      0.64        80

    accuracy                           0.64       239
   macro avg       0.64      0.64      0.64       239
weighted avg       0.64      0.64      0.64       239



# **stage C neural network**

**Neural-1 — Tokenize & pad using P2 text**

In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, LSTM, Bidirectional

MAX_WORDS = 20000
MAX_LEN = 50

tok_nn = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tok_nn.fit_on_texts(Xtr_text)

Xtr_seq = pad_sequences(tok_nn.texts_to_sequences(Xtr_text), maxlen=MAX_LEN, padding="post", truncating="post")
Xte_seq = pad_sequences(tok_nn.texts_to_sequences(Xte_text), maxlen=MAX_LEN, padding="post", truncating="post")

ytr_cat = to_categorical(y_train, num_classes=3)
yte_cat = to_categorical(y_test, num_classes=3)

print("Neural shapes:", Xtr_seq.shape, Xte_seq.shape)


Neural shapes: (954, 50) (239, 50)


# **neural 2 GRU**

In [25]:
model_gru = Sequential([
    Input(shape=(MAX_LEN,)),
    Embedding(input_dim=MAX_WORDS, output_dim=64),
    GRU(64),
    Dense(128, activation="relu"),
    Dense(3, activation="softmax")
])

model_gru.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model_gru.fit(Xtr_seq, ytr_cat, epochs=10, batch_size=32, validation_split=0.1)

loss, acc_gru = model_gru.evaluate(Xte_seq, yte_cat, verbose=0)
pred_gru = np.argmax(model_gru.predict(Xte_seq), axis=1)
mf1_gru = f1_score(y_test, pred_gru, average="macro")

print("GRU Accuracy:", acc_gru)
print("GRU Macro-F1:", mf1_gru)


Epoch 1/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.3555 - loss: 1.0990 - val_accuracy: 0.3125 - val_loss: 1.0989
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.3438 - loss: 1.0973 - val_accuracy: 0.3646 - val_loss: 1.0959
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3695 - loss: 1.0864 - val_accuracy: 0.4167 - val_loss: 1.0864
Epoch 4/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4033 - loss: 1.0483 - val_accuracy: 0.3542 - val_loss: 1.0825
Epoch 5/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4650 - loss: 0.9901 - val_accuracy: 0.4062 - val_loss: 1.2438
Epoch 6/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4837 - loss: 0.9407 - val_accuracy: 0.4583 - val_loss: 1.2368
Epoch 7/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5175 - loss: 0.8675 - val_accuracy: 0.4583 - val_loss: 1.1509
Epoch 8/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5594 - loss: 0.8279 - val_accuracy: 0.4062 - v

# **Neural-3 — BiLSTM model**

In [26]:
model_bilstm = Sequential([
    Input(shape=(MAX_LEN,)),
    Embedding(input_dim=MAX_WORDS, output_dim=64),
    Bidirectional(LSTM(64)),
    Dense(128, activation="relu"),
    Dense(3, activation="softmax")
])

model_bilstm.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model_bilstm.fit(Xtr_seq, ytr_cat, epochs=10, batch_size=32, validation_split=0.1)

loss, acc_bilstm = model_bilstm.evaluate(Xte_seq, yte_cat, verbose=0)
pred_bilstm = np.argmax(model_bilstm.predict(Xte_seq), axis=1)
mf1_bilstm = f1_score(y_test, pred_bilstm, average="macro")

print("BiLSTM Accuracy:", acc_bilstm)
print("BiLSTM Macro-F1:", mf1_bilstm)


Epoch 1/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.3846 - loss: 1.0796 - val_accuracy: 0.4583 - val_loss: 1.0634
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4767 - loss: 0.9831 - val_accuracy: 0.5000 - val_loss: 0.9936
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6585 - loss: 0.7198 - val_accuracy: 0.4583 - val_loss: 1.1741
Epoch 4/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7995 - loss: 0.4900 - val_accuracy: 0.5938 - val_loss: 1.0278
Epoch 5/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9138 - loss: 0.2779 - val_accuracy: 0.5417 - val_loss: 1.1284
Epoch 6/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9744 - loss: 0.1003 - val_accuracy: 0.6042 - val_loss: 1.0996
Epoch 7/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9860 - loss: 0.0503 - val_accuracy: 0.5938 - val_loss: 1.0558
Epoch 8/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9895 - loss: 0.0342 - val_accuracy: 0.6146 - v

# **stage C : Tranformers**

Fine-tune AraBERT

In [29]:

if torch.backends.mps.is_available():
    ft_device = torch.device("mps")
else:
    ft_device = torch.device("cpu")

print("Fine-tuning device:", ft_device)



Fine-tuning device: mps


In [ ]:
'''
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

MODEL_NAME = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert = AutoModel.from_pretrained(MODEL_NAME)
bert.eval()

device = "cpu"   
bert.to(device)

print("AraBERT loaded on", device)
'''

AraBERT loaded on cpu


In [30]:
ft_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
ft_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)
ft_model.to(ft_device)

MAX_LEN_FT = 64
BATCH_FT = 4
EPOCHS_FT = 2
LR_FT = 2e-5

train_enc = ft_tokenizer(Xtr_text, padding=True, truncation=True, max_length=MAX_LEN_FT, return_tensors="pt")
test_enc  = ft_tokenizer(Xte_text, padding=True, truncation=True, max_length=MAX_LEN_FT, return_tensors="pt")

y_train_t = torch.tensor(y_train, dtype=torch.long)
y_test_t  = torch.tensor(y_test, dtype=torch.long)

train_ds = TensorDataset(train_enc["input_ids"], train_enc["attention_mask"], y_train_t)
train_loader = DataLoader(train_ds, batch_size=BATCH_FT, shuffle=True)

optimizer = AdamW(ft_model.parameters(), lr=LR_FT)
loss_fn = nn.CrossEntropyLoss()

ft_model.train()
for epoch in range(EPOCHS_FT):
    total_loss = 0.0
    for input_ids, attn_mask, labels in train_loader:
        input_ids = input_ids.to(ft_device)
        attn_mask = attn_mask.to(ft_device)
        labels = labels.to(ft_device)

        optimizer.zero_grad()
        outputs = ft_model(input_ids=input_ids, attention_mask=attn_mask)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS_FT} - Loss: {total_loss/len(train_loader):.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2 - Loss: 0.9079
Epoch 2/2 - Loss: 0.6363


In [31]:
# Evaluate
ft_model.eval()
with torch.no_grad():
    input_ids = test_enc["input_ids"].to(ft_device)
    attn_mask = test_enc["attention_mask"].to(ft_device)
    logits = ft_model(input_ids=input_ids, attention_mask=attn_mask).logits
    preds_ft = torch.argmax(logits, dim=1).cpu().numpy()

ft_acc = accuracy_score(y_test, preds_ft)
ft_mf1 = f1_score(y_test, preds_ft, average="macro")

print("Fine-tuned AraBERT Accuracy:", ft_acc)
print("Fine-tuned AraBERT Macro-F1:", ft_mf1)
print(classification_report(y_test, preds_ft, target_names=["Negative","Neutral","Positive"]))


Fine-tuned AraBERT Accuracy: 0.6903765690376569
Fine-tuned AraBERT Macro-F1: 0.6924218954615183
              precision    recall  f1-score   support

    Negative       0.76      0.66      0.71        80
     Neutral       0.59      0.73      0.66        79
    Positive       0.76      0.68      0.72        80

    accuracy                           0.69       239
   macro avg       0.70      0.69      0.69       239
weighted avg       0.70      0.69      0.69       239



 Best traditional from Stage C

In [32]:
best_trad = final_df.sort_values("MacroF1", ascending=False).iloc[0]
trad_model = best_trad["Model"]
trad_acc = float(best_trad["Accuracy"])
trad_mf1 = float(best_trad["MacroF1"])




Best neural

In [33]:
if mf1_bilstm > mf1_gru:
    neural_model = "BiLSTM"
    neural_acc = float(acc_bilstm)
    neural_mf1 = float(mf1_bilstm)
else:
    neural_model = "GRU"
    neural_acc = float(acc_gru)
    neural_mf1 = float(mf1_gru)

comparison_df = pd.DataFrame([
    {"Model Family": "Traditional ML", "Best Model": trad_model, "Accuracy": trad_acc, "Macro-F1": trad_mf1},
    {"Model Family": "Neural Network", "Best Model": neural_model, "Accuracy": neural_acc, "Macro-F1": neural_mf1},
    {"Model Family": "Transformer", "Best Model": "AraBERT Fine-tuned", "Accuracy": float(ft_acc), "Macro-F1": float(ft_mf1)},
]).sort_values("Macro-F1", ascending=False).reset_index(drop=True)

comparison_df

,Model Family,Best Model,Accuracy,Macro-F1
0,Transformer,AraBERT Fine-tuned,0.690377,0.692422
1,Traditional ML,SVM,0.640167,0.640270
2,Neural Network,BiLSTM,0.535565,0.487598
